In [1]:
import matplotlib.pyplot as plt
from typing import List, Tuple

**PUNKT 1**
```
Ranking    Wyraz(str).     F[procent] - czestotliwosc.      r x f(słupkami poziomymi)
```
Manuskrypt wojnicza oraz dla dowolnego języka naturalnego



Wybranie odpowiedniego tekstu do przetwarzania. str: [1- 118]


In [2]:
with open("FSG.txt", "r") as file, open('wynik.txt', 'w') as destination:
  for idx, line in enumerate(file):
      if "page" in line or ("#" not in line and "$" not in line):
        destination.write(line)
      if idx == 2679: break

Stworzenie listy zawierającej same słowa
i wyświetlenie ilości słów

In [3]:
t = []
with open("wynik.txt", "r") as wynik:
  for line in wynik:
    if "#" not in line:
      x = line.split(',')
      x[-1] = x[-1].strip()  # usuwa nowe linie
      if '-' in x[-1]  or '=' in x[-1]:
        x[-1] = x[-1][:-1]
      for i in x:
          t.append(i)
len(t)


10767

Policzenie ilości danego słowa w tekscie

In [4]:
counted: List[Tuple[str, int]] = [(unique_word, t.count(unique_word)) for unique_word in filter(lambda word: word, set(t))]


In [6]:
counted.sort(key=lambda x: -x[1])
counted[0]

('8AM', 482)

In [7]:

print("rank|word|ratio|chart")
for idx, (word, occurences) in list(enumerate(counted, 1)):
  ratio: float = occurences/len(t)*100
  y: List[str] = [f'{idx} | {word} | {ratio} | ']
  x = [idx * ratio]
  
  plt.figure(figsize=(3, 0.5))
  plt.barh(y, x)
  
  plt.xlim([0,100])

  plt.xlabel("r x f")
  plt.show()


rank|word|Percent(%)| chart


RuntimeError: In FT2Font: Can not load face (error code 0x55)

<Figure size 300x50 with 1 Axes>

RuntimeError: In FT2Font: Can not load face (error code 0x55)

<Figure size 300x50 with 1 Axes>

...

<Figure size 300x50 with 1 Axes>

RuntimeError: In FT2Font: Can not load face (error code 0x55)

<Figure size 300x50 with 1 Axes>

RuntimeError: In FT2Font: Can not load face (error code 0x55)

<Figure size 300x50 with 1 Axes>

**PUNKT 2**

Sąsiedztwo między wyrazami
Sąsiedztwo to dwa wyrazy do tylu i dwa wyrazy do przodu.
```
W porządku alfabetycznym
Wyr1.           Wyr1
Wyr2.           Wyr2
Wyr3.           Wyr3
Wyr4            Wyr4
Wyr5            Wyr5
```
I robimy strzaleczkami połączenia między wyrazami sąsiadującymi.

Powinien się znaleźć wyraz który ma znacznie dużo sąsiadów - to jest rdzeń języka.



**Punkt 3** : Analiza bigramów

Badać bigramy w ramach akapitów.
```
Wyr1 wyr2 wyr3 wyr4 wyr5 wyr6 ….

Pary sąsiadujących ze sobą wyrazów -2 sąsiednie.

Wyr1   wyr2      [ilosc wystąpień]
Wyr2   wyr3      [ilosc wystąpień]
Wyr3   wyr4      [ilosc wystąpień]
Wyr4   wyr5      [ilosc wystąpień]
```
W manuskrypie większość bigramów będą pojawiające się 1 raz.


